# NAIRU Models

(Non-accelerating inflation rate of unemployment)

## Python set-up

In [1]:
import arviz as az
import numpy as np
import pandas as pd
import pymc as pm


In [2]:
# local imports
from abs_data_capture import (
    metacol,
    get_abs_series, 
    get_abs_data, 
    get_abs_directory,
)

## Data capture

In [3]:
# inflation
inflation_meta, inflation = get_abs_series(cat_id="6401.0", series_ids=["A3604510W"])
trimmed_mean = inflation["A3604510W"].dropna()  # Q/Q percent change, seasonally adjusted

# unemployment rate
lfs_meta, lfs = get_abs_series(cat_id="6202.0", series_ids=["A84423043C", "A84423047L"])   
employed = lfs["A84423043C"]  # employed persons, thousands, seasonally adjusted
labour_force = lfs["A84423047L"]  # labour force, thousands, seasonally adjusted
ue_rate_m = ((1 - employed / labour_force) * 100).dropna()
ue_rate_q = (
    ue_rate_m
    .groupby(pd.PeriodIndex(ue_rate_m.index, freq="Q"))
    .agg(['mean', 'count'])
    .apply(lambda x: x['mean'] if x['count'] == 3 else np.nan, axis=1)
    .dropna()
)

# import prices
trade_meta, trade = get_abs_series(cat_id="6457.0", series_ids=["A2298279F"])
import_prices = trade["A2298279F"].dropna()  # Import Price Index by Balance of Payments, index, original
# Annual growth in consumer inport price deflator
cipd_growth = import_prices.pct_change(periods=4).dropna() * 100  

# quarterly real unit labour costs growth
nat_accounts_meta, nat_accounts = get_abs_series(cat_id="5206.0", series_ids='A2435279C')
rulcg = nat_accounts["A2435279C"].dropna()  # Unit labour cost - Real - Non-farm: Q/Q Percentage, Seas Adj
